In [1]:
import plotly.express as px
from prophet import Prophet
import pandas as pd 
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import datetime
from sklearn.utils import shuffle
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from itertools import product
import itertools, random
import plotly.express as px


pd.set_option('max_column', None)
pd.set_option('max_row', 100)

In [68]:
def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): spread return
    """
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    
    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [95]:
stock_prices_df = pd.concat([
        pd.read_csv("jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
    ])
stock_prices_df["Date"] = pd.to_datetime(stock_prices_df["Date"])

In [96]:
stock_prices_df_full = stock_prices_df[stock_prices_df.Date > pd.to_datetime(max(stock_prices_df.Date)) - timedelta(days=365)] # Last year
stock_prices_df_full["Date"] = pd.to_datetime(stock_prices_df_full["Date"])

stocks = pd.DataFrame(stock_prices_df_full.SecuritiesCode.unique(), columns=['SecuritiesCode'])

stock_prices_df_full['Year'] = stock_prices_df_full.Date.dt.year
stock_prices_df_full['Month'] = stock_prices_df_full.Date.dt.month

# Montly DataFrame
combinazioni_anno_mese = stock_prices_df_full.groupby(['Year', 'Month']).size().reset_index()
dict_mesi = {}
for index, row in combinazioni_anno_mese.iterrows():
    anno = row['Year']
    mese = row['Month']
    dict_mesi[(anno, mese)] = stock_prices_df_full[(stock_prices_df_full.Year == anno) & (stock_prices_df_full.Month == mese)]

stock_prices_df_full

/var/folders/y7/2f0pm2sd7qj4st94l0b5jnmxrw9r2t/T/ipykernel_30994/2079331771.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/y7/2f0pm2sd7qj4st94l0b5jnmxrw9r2t/T/ipykernel_30994/2079331771.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/y7/2f0pm2sd7qj4st94l0b5jnmxrw9r2t/T/ipykernel_30994/2079331771.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Year,Month
1842622,20201204_1301,2020-12-04,1301,2819.0,2819.0,2800.0,2811.0,9200,1.0,NaN,False,0.001779,2020,12
1842623,20201204_1332,2020-12-04,1332,420.0,425.0,420.0,424.0,1672100,1.0,NaN,False,-0.004751,2020,12
1842624,20201204_1333,2020-12-04,1333,2200.0,2224.0,2184.0,2212.0,181700,1.0,NaN,False,-0.014652,2020,12
1842625,20201204_1375,2020-12-04,1375,1778.0,1786.0,1740.0,1742.0,148300,1.0,NaN,False,0.019837,2020,12
1842626,20201204_1376,2020-12-04,1376,1548.0,1548.0,1511.0,1532.0,2300,1.0,NaN,False,-0.013201,2020,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,2021,12
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,2021,12
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,2021,12
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,2021,12


In [97]:
NUM_SIMULATIONS = 100000

dict_dates = {}
i = 0
max_date = pd.to_datetime(max(stock_prices_df_full.Date))

for data_temp in stock_prices_df_full.Date.unique():
    start_date = pd.to_datetime(data_temp)
    stock_prices_df_full_w_rank = stock_prices_df_full[(stock_prices_df_full.Date == start_date)]
    dict_dates[start_date] = {}

    # Day simulations with shuffle
    for x in tqdm(range(NUM_SIMULATIONS)):
        stocks_shuffled = shuffle(stocks.copy()).reset_index(drop=True)
        stocks_shuffled['Rank'] = stocks_shuffled.index.astype(int)
        stock_prices_df_full_w_rank_2 = stock_prices_df_full_w_rank.copy().merge(stocks_shuffled, left_on = 'SecuritiesCode', right_on = 'SecuritiesCode')

        dict_dates[start_date][x] = (stocks_shuffled,_calc_spread_return_per_day(stock_prices_df_full_w_rank_2, 200, 2))
    
    spread_values = [v[1] for v in dict_dates[start_date].values()]

    i += 1

    #break
    
print(i)

  0%|          | 0/100000 [00:00<?, ?it/s]

AssertionError: 

In [98]:
# Spread distribution example in one day
fig = px.histogram(spread_values)
fig.show()